<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Canales%20de%20Youtube/Python%20Engineer/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naive Bayes**

Particularmente, este algoritmo no lo conocia, y por lo que he visto hasta ahora funciona como un **clasificador** basandose principalmente en el **teorema de bayes**.

**Teorema de bayes**

$$
P(A/B) = \frac{P(B/A) \cdot P(A)}{P(B)}
$$

Eso si, para aprovechar este teorema es que se tiene que cumplir la condicion de que los atributos o **componentes del vector X sean independientes entre si (Se asume que los eventos son independientes)**.

$$
P(y/X) = \frac{P(X/y) \cdot P(y)}{P(X)} = \frac{P(x_{1}/y) \quad ... \quad P(x_{n}/y) \cdot P(y)}{P(X)}
$$

Asi, luego la manera de escoger a que clasificacion pertenece el vector X, es calculando todas las probabilidades condicionales (**Nota**: el $P(x)$ lo podemos omitir ya que va a estar presente en todas las ecuaciones)

$
$

$
y = argmax_{y} \quad P(x_{1}/y) \quad ... \quad P(x_{n}/y) \cdot P(y)
$

$
y = argmax_{y} \quad log(P(x_{1}/y)) + \quad ... \quad + log(P(x_{n}/y)) + log(P(y))
$

$
$

**Por ultimo, nos falta definir:**

$
$

$P(y)$: Frecuencia (cantidad de veces que esta presente la clasificacion y en los datos)

$
P(x_{i}/y) = \frac{1}{\sqrt{2\pi \sigma_{y}^{2}}} \cdot e^{(-\frac{(x_{i} - \mu_{y})^2}{2σ_{y}^{2}})}
$

# **Armando el algoritmo desde cero**

In [9]:
import numpy as np

class NaiveBayes:

  def fit(self, X, y):
    n_samples, n_features = X.shape
    self._classes = np.unique(y)
    n_classes = len(self._classes)

    # Ahora calculamos la media y varianza de cada componente de X (features)
    self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
    self._var = np.zeros((n_classes, n_features), dtype=np.float64)
    self._priors = np.zeros(n_classes, dtype=np.float64)

    for c in self._classes:
      X_c = X[c == y]
      self._mean[c, :] = X_c.mean(axis=0)
      self._var[c, :] = X_c.var(axis=0)
      self._priors[c] = X_c.shape[0] / float(n_samples)

  def predict(self, X):
    y_pred = [self._predict(x) for x in X]
    return y_pred

  def _predict(self, x):
    posteriors = []

    for idx, c in enumerate(self._classes):
      prior = np.log(self._priors[idx])
      class_conditional = np.sum(np.log(self._pdf(idx, x)))
      posterior = prior + class_conditional
      posteriors.append(posterior)

    return self._classes[np.argmax(posteriors)]

  def _pdf(self, class_idx, x): # Probability Density Function
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    numerator = np.exp(-(x - mean)**2 / (2*var))
    denominator = np.sqrt(2*np.pi*var)
    return numerator/denominator


## **Ahora probemos el algortimo**

In [2]:
# Importamos las bibliotecas
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt


In [3]:
# Definimos una funcion para medir el acierto en las predicciones
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true == y_pred) / len(y_true)
  return accuracy

In [10]:
# Generamos y ordenamos los sets de entrenamiento y de validacion
X, y = datasets.make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Cargamos y entrenamos al algortimo
nb = NaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

# Medimos el acierto que tuvo
acc = accuracy(y_test, predictions)
acc

0.965

## **Ahora probemos con el algoritmo de sklearn**

In [12]:
# Importamos el algoritmo
from sklearn.naive_bayes import GaussianNB

In [13]:
# Cargamos y entrenamos al algoritmo
clf = GaussianNB()
clf.fit(X_train, y_train)

# Generamos las predicciones sobre el set de validacion
predictions = clf.predict(X_test)

In [14]:
# Medimos el acierto que tuvo
acc_sk = accuracy(y_test, predictions)
acc_sk

0.965